In [ ]:
from datetime import datetime

import camelot
import pandas as pd

In [ ]:
## Bank Name: Standard Chartered Bank Nigeria

In [ ]:
# Configuration
PDF_FILE_PATH = '../docs/sc-statement-2.pdf'
PDF_PASSWORD = '065403'
INPUT_DATE_FORMAT = "%d %b %Y"
OUTPUT_DATE_FORMAT = "%m/%d/%Y"

# Column Headers from the Document
COLUMN_NAMES = [
    "Date",
    "Description",
    "Deposit",
    "Withdrawal",
    "Balance"
]

def extract_and_clean_transactions(pdf_path: str, password: str) -> pd.DataFrame:
    """Extract, build, and clean transaction data from a PDF."""
    print("Starting extraction...")
    tables = camelot.read_pdf(pdf_path, pages='all', flavor='lattice', password= password)

    num_columns = len(COLUMN_NAMES)

    transactions_df = pd.concat([table.df for table
                                 in tables
                                 if table.df.shape[1] == num_columns], ignore_index=True)
    transactions_df.dropna(axis=1, how='all', inplace=True)

    print(f"Initial DataFrame shape: {transactions_df.shape}")
    display(transactions_df.head(50))
    display(transactions_df.tail(50))

    transactions_df.columns = COLUMN_NAMES[:num_columns]

    print("Performing final data cleaning...")
    cleaned_df = transactions_df.copy()

    # Clean up newline characters and leading/trailing spaces
    for col in cleaned_df.columns:
        if cleaned_df[col].dtype == 'object':
            cleaned_df.loc[:, col] = (
                cleaned_df[col]
                .str.replace('\n', ' ', regex=False)
                .str.strip()
            )

    def clean_numeric_col(series: pd.Series) -> pd.Series:
        # Remove commas, letters, and any non-numeric characters except for '.' and '-'
        cleaned_series = series.astype(str).str.replace(',', '', regex=False)
        cleaned_series = cleaned_series.str.replace(r'[^\d.-]', '', regex=True)
        return pd.to_numeric(cleaned_series, errors='coerce')

    cleaned_df.loc[:, 'Deposit'] = clean_numeric_col(cleaned_df['Deposit'])
    cleaned_df.loc[:, 'Withdrawal'] = clean_numeric_col(cleaned_df['Withdrawal'])
    cleaned_df.loc[:, 'Balance'] = clean_numeric_col(cleaned_df['Balance'])

    cleaned_df.loc[:, 'Date'] = pd.to_datetime(
        cleaned_df['Date'],
        format=INPUT_DATE_FORMAT,
        errors='coerce'
    )

    cleaned_df = cleaned_df[cleaned_df['Date'].notna()]
    cleaned_df.reset_index(drop=True, inplace=True)

    print(f"Cleaned DataFrame shape: {transactions_df.shape}")
    display(cleaned_df.head(50))
    display(cleaned_df.tail(50))

    print("Extraction and cleaning complete.")
    return cleaned_df

In [ ]:
if __name__ == '__main__':
    # --- Execute the script ---
    final_transactions_df = extract_and_clean_transactions(PDF_FILE_PATH, PDF_PASSWORD)

    # --- Display Results ---
    print("\n--- Extracted Transactions ---")
    print(f"Total transactions extracted: {len(final_transactions_df)}")

    # Display the first 10 and last 10 transactions
    print("\nFirst 50 Transactions:")
    display(final_transactions_df.head(50))

    print("\nLast 50 Transactions:")
    display(final_transactions_df.tail(50))

    # --- Optional: Save to CSV ---
    try:
        output_csv_path = f'../output/sc_extracted_transactions{datetime.now().strftime('%d%m%Y%H%M%S')}.csv'
        final_transactions_df.to_csv(output_csv_path, date_format=OUTPUT_DATE_FORMAT, index=False)
        print(f"\nSuccessfully saved cleaned data to '{output_csv_path}'")
    except Exception as e:
        print(f"\nCould not save to CSV: {e}")